## **I. Thu thập dữ liệu (Crawling Data Method)**

### 1. Thu thập dữ liệu trên **Batdongsan.com**

In [30]:
# We can install selenium by run the segment code below
# !pip install selenium


In [5]:
# Import necessary librarys
import time
import csv
import logging
import pandas as pd
import requests
import tempfile
import os
import random


from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from multiprocessing import Pool, current_process, log_to_stderr, get_logger
from multiprocessing.pool import ThreadPool


In [32]:
# Config logging 
log_to_stderr()
logger = get_logger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [33]:
def write_to_csv(details):
    csv_file = 'property_listings.csv'

    file_empty = False
    try:
        with open(csv_file, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            file_empty = len(list(reader)) == 0
    except FileNotFoundError:
        file_empty = True

    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if file_empty:
            writer.writerow([
                'Title', 'Price', 'Address', 'Price per m2', 'Area', 
                'Bedrooms', 'Toilets', 'Published At', 
                'Diện tích', 'Mức giá', 'Hướng nhà', 'Hướng ban công',
                'Ngày đăng', 'Ngày hết hạn', 'Loại tin', 'Mã tin',
                'Project Title', 'Status', 'Number of Apartments',
                'Number of Buildings', 'Developer', 'Project Address'
            ])

        row = [
            details.get('Project Title', ''),
            details.get('Mức giá', ''),
            details.get('Project Address', ''),
            details.get('Price per m2', ''),
            details.get('Diện tích', ''),
            details.get('Số phòng ngủ', ''),
            details.get('Số toilet', ''),
            details.get('Ngày đăng', ''),
            details.get('Diện tích', ''),
            details.get('Mức giá', ''),
            details.get('Hướng nhà', ''),
            details.get('Hướng ban công', ''),
            details.get('Ngày đăng', ''),
            details.get('Ngày hết hạn', ''),
            details.get('Loại tin', ''),
            details.get('Mã tin', ''),
            details.get('Project Title', ''),
            details.get('Status', ''),
            details.get('Number of Apartments', ''),
            details.get('Number of Buildings', ''),
            details.get('Developer', ''),
            details.get('Project Address', '')
        ]
        writer.writerow(row)

In [34]:
def create_driver():
    firefox_options = Options()
    firefox_options.add_argument("--headless")
    firefox_options.add_argument("--no-sandbox")
    firefox_options.add_argument("--disable-dev-shm-usage")
    firefox_options.binary_location = "C:\\Program Files\\Mozilla Firefox\\firefox.exe"
    service = Service('./geckodriver-v0.35.0-win32/geckodriver.exe')
    return webdriver.Firefox(service=service, options=firefox_options)

In [35]:
def safe_find_element(driver, locator, retries=3):
    for attempt in range(retries):
        try:
            return driver.find_element(*locator)
        except StaleElementReferenceException:
            time.sleep(1)
        except NoSuchElementException:
            logging.warning("Element not found.")
            break
        except Exception as e:
            logging.error(f"Error finding element: {str(e)}")
            break
    return None

In [36]:
def scrape_detail_page(url):
    driver = create_driver()
    property_details = {}
    project_info = {}

    try:
        driver.get(url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 're__pr-specs-content'))
        )
        logging.info(f"Accessed URL: {url}, Page Title: {driver.title}")

        specs = driver.find_elements(By.CLASS_NAME, 're__pr-specs-content-item')

        for spec in specs:
            title_element = safe_find_element(spec, (By.CLASS_NAME, 're__pr-specs-content-item-title'))
            value_element = safe_find_element(spec, (By.CLASS_NAME, 're__pr-specs-content-item-value'))
            
            title = title_element.text.strip() if title_element else 'N/A'
            value = value_element.text.strip() if value_element else 'N/A'
            
            property_details[title] = value

        project_card = safe_find_element(driver, (By.CLASS_NAME, 're__project-card-info'))

        if project_card:
            project_title = safe_find_element(project_card, (By.CLASS_NAME, 're__project-title')).text.strip()
            project_info['Project Title'] = project_title

            project_status = safe_find_element(project_card, (By.CLASS_NAME, 're__long-text')).text.strip()
            project_info['Status'] = project_status

            apartments = project_card.find_elements(By.CLASS_NAME, 're__prj-card-config-value')
            if len(apartments) > 1:
                project_info['Number of Apartments'] = apartments[1].text.strip()
            if len(apartments) > 2:
                project_info['Number of Buildings'] = apartments[2].text.strip()

            developer_name = project_card.find_elements(By.CLASS_NAME, 're__long-text')
            if len(developer_name) > 1:
                project_info['Developer'] = developer_name[1].text.strip()

        short_info = driver.find_elements(By.CLASS_NAME, 're__pr-short-info-item')

        for info in short_info:
            title = safe_find_element(info, (By.CLASS_NAME, 'title')).text.strip()
            value = safe_find_element(info, (By.CLASS_NAME, 'value')).text.strip()
            property_details[title] = value

        address_element = safe_find_element(driver, (By.CLASS_NAME, 'js__pr-address'))
        if address_element:
            property_details['Project Address'] = address_element.text.strip()

        property_details.update(project_info)
        for title, value in property_details.items():
            logging.info(f"{title}: {value}")

        write_to_csv(property_details)

    except Exception as e:
        logging.error(f"Error accessing page {url}: {str(e)}")
        
    driver.quit()
    return property_details

In [37]:
def scrape_page(url):
    driver = create_driver()
    data = []
    try:
        driver.get(url)
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 're__srp-list')))
        logging.info(f"Accessed URL: {url}, Page Title: {driver.title}")

        while True:
            try:
                listings = driver.find_elements(By.CLASS_NAME, 'js__card')
                logging.info(f"Number of listings found: {len(listings)}")

                if not listings:
                    logging.info(f'Found 0 listings on {url}')
                    break

                for listing in listings:
                    try:
                        detail_link = listing.find_element(By.TAG_NAME, 'a').get_attribute('href')
                        logging.info(f"Accessing detail URL: {detail_link}")

                        detail_data = scrape_detail_page(detail_link)
                        if detail_data:
                            data.append(detail_data)

                    except StaleElementReferenceException:
                        logging.warning("Stale element reference encountered while accessing detail link.")
                        break

                    except Exception as e:
                        logging.error(f"Error parsing listing: {e}")

                break

            except StaleElementReferenceException:
                logging.warning("Stale element reference encountered while fetching listings. Retrying...")
                time.sleep(1)

    except TimeoutException:
        logging.error(f"Timeout while accessing page: {url}")
    except Exception as e:
        logging.error(f"Error accessing {url}: {str(e)}")
    finally:
        driver.quit()

    return data

In [38]:
def worker(page_number):
    base_url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p'
    url = f'{base_url}{page_number}'
    logging.info(f'Process {current_process().name} scraping page {page_number}...')
    page_data = scrape_page(url)
    time.sleep(random.randint(3, 7))



In [39]:
# Số lượng trang cần xử lý
num_pages = 5

# Số luồng
num_threads = 2

In [ ]:


with ThreadPool(num_threads) as pool:  # Start with 2 processes, increase if stable
    pool.map(worker, range(1, num_pages + 1))

logging.info('Data saved to property_listings.csv')

### 2. Thu thập dữ liệu về giá khuyến nghị của chính phủ 

In [1]:
def download_excel(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx")
        with open(temp_file.name, 'wb') as file:
            file.write(response.content)
        print("File downloaded successfully.")
        return temp_file.name
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        return None


In [2]:
def extract_to_csv(excel_path, output_csv):
    excel_data = None
    try:
        excel_data = pd.ExcelFile(excel_path)
        combined_data = pd.DataFrame()
        
        for sheet_name in excel_data.sheet_names:
            # Load sheet and skip rows until headers
            df = excel_data.parse(sheet_name, skiprows=6)

            print("Crawling...", sheet_name)
            # Check if DataFrame has the expected number of columns
            if df.shape[1] < 6:  # Fewer columns than expected
                print(f"Skipping sheet '{sheet_name}' - Fewer columns than expected.")
                continue
            
            # Rename columns if they match the expected structure
            df.columns = df.columns[:6].tolist() + ['extra_column' for _ in range(df.shape[1] - 6)]
            df = df.rename(columns={df.columns[1]: 'TÊN ĐƯỜNG', df.columns[5]: 'Giá đất đề nghị điều chỉnh'})
            
            # Drop rows without a street name
            df = df.dropna(subset=['TÊN ĐƯỜNG'])
            
            # Convert 'Giá đất đề nghị điều chỉnh' to numeric, coerce errors
            df['Giá đất đề nghị điều chỉnh'] = pd.to_numeric(df['Giá đất đề nghị điều chỉnh'], errors='coerce')
            
            # Drop rows with NaN values in 'Giá đất đề nghị điều chỉnh' after conversion
            df = df.dropna(subset=['Giá đất đề nghị điều chỉnh'])
            
            # Add district column
            df['QUẬN'] = sheet_name
            
            # Clone "THỦ ĐỨC" to "Quận 2" and "Quận 9"
            if 'THỦ ĐỨC' in df['QUẬN'].values:
                df_2 = df[df['QUẬN'] == 'Thủ Đức'].copy()
                df_2['QUẬN'] = 'Quận 2'  # Clone for Quận 2
                df_9 = df[df['QUẬN'] == 'Thủ Đức'].copy()
                df_9['QUẬN'] = 'Quận 9'  # Clone for Quận 9
                df = pd.concat([df, df_2, df_9], ignore_index=True)
            
            # Select relevant columns and ignore extra columns
            df_filtered = df[['TÊN ĐƯỜNG', 'Giá đất đề nghị điều chỉnh', 'QUẬN']]
            combined_data = pd.concat([combined_data, df_filtered], ignore_index=True)
        
        if not combined_data.empty:
            # Group by 'TÊN ĐƯỜNG' and 'District' and calculate the mean of 'Giá đất đề nghị điều chỉnh'
            aggregated_data = combined_data.groupby(['TÊN ĐƯỜNG', 'QUẬN'], as_index=False)['Giá đất đề nghị điều chỉnh'].mean()
            
            # Sort the aggregated data by 'District' and then by 'Giá đất đề nghị điều chỉnh' in descending order
            aggregated_data = aggregated_data.sort_values(by=['QUẬN', 'Giá đất đề nghị điều chỉnh'], ascending=[True, False])
            
            # Save the sorted aggregated data to the CSV file
            aggregated_data.to_csv(output_csv, index=False)
            print(f"Data saved to {output_csv}")
        else:
            print("No data found with the specified columns.")
    except ImportError as e:
        print("Missing required module. Please install the required module and try again.")
        print(f"Error: {e}")
    finally:
        if excel_data is not None:
            excel_data.close()  # Ensure the file is closed after reading


In [9]:
def main():
    url = "https://xdcs.cdnchinhphu.vn/446259493575335936/2024/10/22/phu-luc-bang-8-bgd-dat-o-1-1729562018876764836390-1-17295692360131820457301.xlsx"
    output_csv = "gbd.csv"
    
    excel_path = download_excel(url)
    if excel_path:
        try:
            extract_to_csv(excel_path, output_csv)
        finally:
            os.remove(excel_path)  # Clean up the file


In [ ]:
if __name__ == "__main__":
    main()
